# Left off here, 2017-06-09

* for all but 81 links, was able to split tags containing "." into individual atomic tags.
* this reduced tag count from 1634 to 1553
* for the 81 bad links, try this:
    * look at each individually
    * what is the common problem, if any?
        * private?
        * non-shared?
        * cached by pinboard?
        * tag syntax?
        * other?

In [6]:
import sys
import pinboard
from ruamel.yaml import YAML
from time import sleep
from random import sample, random

In [7]:
yaml=YAML(typ='safe')  

In [8]:
with open("data/keys/pbkey.yaml") as kh:
    fin = kh.read()
    config = yaml.load(fin)

In [10]:
print(type(config))
mykey = config.get('uname') + ":" + config.get("key")
# print mykey

<class 'dict'>


In [11]:
pb = pinboard.Pinboard(config.get('uname') + ":" + config.get("key"))

In [12]:
type(pb)

pinboard.pinboard.Pinboard

In [13]:
pb.posts.update()

datetime.datetime(2017, 12, 18, 15, 12, 24)

In [14]:
tags_pb = pb.tags.get()

In [16]:
print(type(tags_pb))
print(len(tags_pb))
print(tags_pb[10])

<class 'list'>
1986
<Tag name="&amp;amp;class" count=3>


In [17]:
posts_pb = pb.posts.all()

In [18]:
print(type(posts_pb))
print(len(posts_pb))
print(posts_pb[10])
print(type(posts_pb[10]))
print(posts_pb[10].tags)

<class 'list'>
2091
<Bookmark description="b'The State of Javascript, 2017'" url="stateofjs.com">
<class 'pinboard.pinboard.Bookmark'>
['useful', 'javascript', '_coms', 'webdev', 'interactive']


In [20]:
a_post = posts_pb[100]
print(a_post)
# dir(a_post)

<Bookmark description="b'soft question - Mathematical games interesting to both you and a 5+-year-old child - MathOverflow'" url="mathoverflow.net">


## Make a tag list

In [21]:
tags = [t.name for t in tags_pb]
print(len(tags))

1986


In [22]:
sample(tags, 20)

['trello',
 'ms-powerpoint',
 'inference',
 '&amp;amp;advice',
 'decision',
 'regs',
 'cmbs',
 'debug',
 'anthropology',
 'vcv',
 'r',
 'worth-of-information',
 'keys',
 'gui',
 'loudness',
 '&amp;bio',
 'feature-hashing',
 'kernels',
 'aml',
 'growth']

In [23]:
acount = 0
ocount = 0
printmax = 10
atomic_tags = []
other_tags = []
for t in tags:
    if "." in t:
        t = t.replace("&amp;amp;", "&")
        t = t.replace("&amp;", "&")
        t = t.replace("$", "&")
        splitto = [str(tg) for tg in t.split(".") if len(tg) > 0] 
        atomic_tags += splitto     
        if acount < printmax and random() < 0.1:
            print("Atomic", t, splitto)
            acount += 1
    else:
        if ocount < printmax and random() < 0.05:
            stag = str(t)
            stag = stag.replace("&amp;amp;", "&")
            stag = stag.replace("&amp;", "&")
            stag = stag.replace("$", "&")
            print("Other", [stag])
            other_tags += stag
            ocount += 1
tag_set = list(set(atomic_tags + other_tags))
tag_set = sorted(tag_set)

Other ['&guide']
Other ['&utility']
Other ['&review']
Other ['&util']
Other ['&explainer']
Other ['&qotd']
Other ['@do(mac)']
Other ['@project(time-series-classification)']
Other ['Hyndman']
Other ['RPA']
Atomic _coms.documentation ['_coms', 'documentation']
Atomic _coms.lang.bash ['_coms', 'lang', 'bash']
Atomic _coms.parallel ['_coms', 'parallel']
Atomic _data.census ['_data', 'census']
Atomic _dmml.eda ['_dmml', 'eda']
Atomic _dmml.industry ['_dmml', 'industry']
Atomic _econ.policy ['_econ', 'policy']
Atomic _sci.astro.pluto ['_sci', 'astro', 'pluto']
Atomic _stat.control-chart ['_stat', 'control-chart']
Atomic _stat.r.sem ['_stat', 'r', 'sem']


In [24]:
print(len(tags), "found tags.  Unique->", len(set(tags)))
print(len(atomic_tags), "atomic tags.  Unique->", len(set(atomic_tags)))
print(len(other_tags), "other tags.  Unique->", len(set(other_tags)))
print(len(tag_set))

1986 found tags.  Unique-> 1986
256 atomic tags.  Unique-> 119
95 other tags.  Unique-> 31
149


In [25]:
sample(tag_set,20)

['H',
 'js',
 'random-forest',
 'scipy',
 'dataframe',
 'd',
 'smoothing',
 '@',
 'g',
 'pivottable',
 'causal',
 'poetry',
 'gam',
 'i',
 'regex',
 'a',
 's',
 'astro',
 'tools',
 'bash']

# Now convert tags to atomic

### define conversion function

In [29]:
def tag_converter(tag_list):
    """convert a list of decorated xx.xx.xx tags to atomic tags"""
    assert type(tag_list) == list
    atomic_tags = []
    for t in tag_list:
        if "." in t:
            splitto = [str(tg).encode("ascii", "ignore") for tg in t.split(".") if len(tg) > 0] 
            atomic_tags += splitto 
        # elif "@" in t:
        #     tgood = t.replace("@", "%")
        #     tgood = tgood.encode("ascii", "ignore")
        #     atomic_tags.append(tgood)
        else:
            tt = t.encode("ascii", "ignore")
            atomic_tags.append(tt)
    atomic_tags = list(set(atomic_tags))
    return atomic_tags

In [30]:
# test conversion
tag_converter([u'_app.adobe-animate', u'animation'])

[b'animation', b'adobe-animate', b'_app']

### test the conversion function

In [34]:
tag_converter([u'_app.adobe-animate', u'animation'])

[b'animation', b'adobe-animate', b'_app']

In [37]:
assert(tag_converter(["_app.autohotkey"])) == ['_app', 'autohotkey']
assert(tag_converter([u".watch"])) ==  ['watch']
assert(tag_converter([u'_app.adobe-animate', u'animation'])) == ['_app', 'animation', 'adobe-animate']
# assert(tag_converter([u'wise', u'_lit.poetry.life', u'life', u'@share', u'awesome'])) ==\
#     ['life', '%share', 'awesome', 'poetry', 'wise', '_lit']

AssertionError: 

In [39]:
tags = [t.name for t in tags_pb]
print(len(tags))

1986


In [41]:
for t in tags:
    try:
        dump = tag_converter(t)
    except:
        print("O SHIT! -> ", t)
        

O SHIT! ->  $chuckable
O SHIT! ->  $chuckable(2016)
O SHIT! ->  $read-often
O SHIT! ->  $rr
O SHIT! ->  &amp;amp;advice
O SHIT! ->  &amp;amp;article
O SHIT! ->  &amp;amp;blog
O SHIT! ->  &amp;amp;book
O SHIT! ->  &amp;amp;book-review
O SHIT! ->  &amp;amp;cheat-sheets
O SHIT! ->  &amp;amp;class
O SHIT! ->  &amp;amp;course
O SHIT! ->  &amp;amp;documentation
O SHIT! ->  &amp;amp;explainer
O SHIT! ->  &amp;amp;guide
O SHIT! ->  &amp;amp;how-to
O SHIT! ->  &amp;amp;inspiration
O SHIT! ->  &amp;amp;list
O SHIT! ->  &amp;amp;long-article
O SHIT! ->  &amp;amp;manual
O SHIT! ->  &amp;amp;merchant
O SHIT! ->  &amp;amp;op_inst
O SHIT! ->  &amp;amp;paper
O SHIT! ->  &amp;amp;pdf
O SHIT! ->  &amp;amp;prank
O SHIT! ->  &amp;amp;present
O SHIT! ->  &amp;amp;presentation
O SHIT! ->  &amp;amp;primer
O SHIT! ->  &amp;amp;project
O SHIT! ->  &amp;amp;r-package
O SHIT! ->  &amp;amp;reading-list
O SHIT! ->  &amp;amp;recipe
O SHIT! ->  &amp;amp;reference
O SHIT! ->  &amp;amp;repo
O SHIT! ->  &amp;amp;review

O SHIT! ->  agressive
O SHIT! ->  ai
O SHIT! ->  air
O SHIT! ->  air-traffic-control
O SHIT! ->  aircraft
O SHIT! ->  airplanes
O SHIT! ->  alerts
O SHIT! ->  algebra
O SHIT! ->  algebraic-geometry
O SHIT! ->  algorithmic
O SHIT! ->  alias
O SHIT! ->  all-modules
O SHIT! ->  alphavantage
O SHIT! ->  altair
O SHIT! ->  alternative-data
O SHIT! ->  alteryx
O SHIT! ->  altman
O SHIT! ->  amazon
O SHIT! ->  ambiance
O SHIT! ->  aml
O SHIT! ->  anaconda
O SHIT! ->  analog
O SHIT! ->  analysis
O SHIT! ->  anchor
O SHIT! ->  animation
O SHIT! ->  anime
O SHIT! ->  anime-js
O SHIT! ->  annotation
O SHIT! ->  anomaly-detection
O SHIT! ->  anomaly_detection
O SHIT! ->  anonymity
O SHIT! ->  anova
O SHIT! ->  anthropology
O SHIT! ->  anti-patterns
O SHIT! ->  api
O SHIT! ->  apl
O SHIT! ->  app-dev
O SHIT! ->  apple
O SHIT! ->  applescript
O SHIT! ->  apply-functions
O SHIT! ->  architecture
O SHIT! ->  archiving
O SHIT! ->  arduino
O SHIT! ->  ares
O SHIT! ->  argmax
O SHIT! ->  argmin
O SHIT! -

O SHIT! ->  microsoft-azure
O SHIT! ->  min
O SHIT! ->  mind-map
O SHIT! ->  mindmap
O SHIT! ->  minecraft
O SHIT! ->  mini-computer
O SHIT! ->  minilog
O SHIT! ->  mining
O SHIT! ->  miso-soup
O SHIT! ->  missing-values
O SHIT! ->  mistakes
O SHIT! ->  mixed-effects-models
O SHIT! ->  miyazaki
O SHIT! ->  mo
O SHIT! ->  mobile-phone
O SHIT! ->  mod
O SHIT! ->  model-evaluation
O SHIT! ->  modeling
O SHIT! ->  models-as-a-service
O SHIT! ->  modules
O SHIT! ->  monetization
O SHIT! ->  money
O SHIT! ->  monotonic
O SHIT! ->  monte-carlo
O SHIT! ->  mortgage
O SHIT! ->  mosh
O SHIT! ->  mosteller
O SHIT! ->  mouse
O SHIT! ->  move
O SHIT! ->  movie
O SHIT! ->  mozart
O SHIT! ->  mozilla
O SHIT! ->  ms-powerpoint
O SHIT! ->  ms-word
O SHIT! ->  msg
O SHIT! ->  mta
O SHIT! ->  multi-class
O SHIT! ->  multimeter
O SHIT! ->  multivariate
O SHIT! ->  muse
O SHIT! ->  musescore
O SHIT! ->  music
O SHIT! ->  musk
O SHIT! ->  mutate
O SHIT! ->  my-stuff
O SHIT! ->  myth
O SHIT! ->  myth-bust
O 

## now apply conversion function

In [277]:
tcount = 0
for post in posts_pb[1000:1100]:
    # 900-1000 are bad!
    # print post
    # print post.tags
    new_tags = tag_converter(post.tags)
    # print new_tags
    post.tags = new_tags
    try:
        post.save()
        tcount += 1
        if tcount%10 == 0:
            print tcount,
    except:
        print "Argh, die", post

Argh, die <Bookmark description="How Well-Structured Is Our Federal Reserve, Anyway?" url="www.bradford-delong.com">
Argh, die <Bookmark description="Exploratory computing with Python" url="mbakker7.github.io">
Argh, die <Bookmark description="Principal Component Analysis on Images" url="blog.thehackerati.com">
Argh, die <Bookmark description="pandashells pandas on the command line" url="github.com">
Argh, die <Bookmark description="Termtile OS X scripts to tile the terminal window" url="github.com">
Argh, die <Bookmark description="List of Lists on Github - awesome" url="github.com">
Argh, die <Bookmark description="mltframework/shotcut · GitHub" url="github.com">
10 Argh, die <Bookmark description="How a Kalman filter works, in pictures" url="www.bzarg.com">
Argh, die <Bookmark description="szilard/benchm-ml" url="github.com">
Argh, die <Bookmark description="How Do You Know if Your Data Has Signal?" url="feedproxy.google.com">
Argh, die <Bookmark description="Documentation at scale:

In [278]:
posts_pb = pb.posts.all()

In [279]:
# second pass to clean up misses
pcount = 0
for post in posts_pb:
    current_tags = post.tags
    dot_check = ["." in t for t in current_tags]
    if any(dot_check):
        pcount += 1
        if pcount%25 == 0:
            sleep(0.5)
            #print "z",
        # print "\n", post, "\n", current_tags, "\n", 
        new_tags = tag_converter(current_tags)
        # print new_tags
        try:
            post.tags = new_tags
            post.save()
            print "smile",
            sleep(1)
        except:
            print "ARGH"
            print post
            print current_tags
            print new_tags
            print " "
            sleep(0.25)
    # else:
    #    print ".",
print pcount

ARGH
<Bookmark description="The Vision of Sin, by Alfred Tennyson" url="www.blackcatpoems.com">
[u'wise', u'_lit.poetry', u'death', u'life', u'@share', u'awesome']
['life', 'death', 'wise', 'awesome', 'poetry', '@share', '_lit']
 
ARGH
<Bookmark description="Building Interactive Dashboards with Jupyter" url="blog.dominodatalab.com">
[u'_coms.lang.python.ipython-notebook', u'_coms.lang.python.jupyter', u'interactive', u'_dmml', u'reactive']
['_coms', 'lang', 'jupyter', 'python', 'reactive', '_dmml', 'ipython-notebook', 'interactive']
 
ARGH
<Bookmark description="How far away is Mars?" url="feeds.boingboing.net">
[u'_coms.project.orery', u'_sci.astro.mars', u'_sci.astro', u'_sci.astro.orery']
['_coms', 'astro', 'project', 'orery', '_sci', 'mars']
 
ARGH
<Bookmark description="Interactive Scatterplots of Census Data Using ggvis" url="feedproxy.google.com">
[u'_data.census', u'swc']
['_data', 'census', 'swc']
 
ARGH
<Bookmark description="Scipy Lecture Notes¶" url="www.scipy-lectures.org"

In [257]:
pb.posts.get(url="www.scipy-lectures.org")

{u'date': datetime.datetime(2017, 6, 9, 0, 36, 46),
 u'posts': [<Bookmark description="Practical API Versioning" url="fly.io">,
  <Bookmark description="Practical API Versioning Hacker News Comments" url="news.ycombinator.com">,
  <Bookmark description="ruyaml python yaml alternative" url="yaml.readthedocs.io">,
  <Bookmark description="Doodling in Math Class: Spirals, Fibonacci, and Being a Plant [2 of 3]" url="vimeo.com">,
  <Bookmark description="HTML5 Animation Tools" url="noeticforce.com">,
  <Bookmark description="VI Hart Vimeo Chanel" url="vimeo.com">,
  <Bookmark description="Hexaflexagons" url="vimeo.com">],
 u'user': u'MuttonSea'}